In [9]:
!pip install tensorflow 
!pip install keras 
!pip install pickle
!pip install nltk
!pip install flask
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

ERROR: Could not find a version that satisfies the requirement pickle
ERROR: No matching distribution found for pickle


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mayekar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mayekar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


205 documents
23 classes ['Offers', 'QueriesandFeedback', 'TrackCurrentOrder', 'complaint', 'contact', 'contactstore', 'createaccount', 'general', 'goodbye', 'greeting', 'help', 'hours', 'identity', 'menu', 'negative_feedback', 'offers', 'positive_feedback', 'recipe_enquiry', 'sanitation', 'storesnearme', 'suggest', 'veg_enquiry', 'vegan_enquiry']
249 unique lemmatized words [',', '.', 'a', 'about', 'account', 'ahead', 'all', 'also', 'am', 'amazing', 'an', 'and', 'any', 'are', 'available', 'awesome', 'bad', 'baked', 'behavior', 'best', 'bot', 'brownie', 'buy', 'bye', 'call', 'can', 'care', 'changer', 'cheesy', 'choco', 'choose', 'clean', 'cleanliness', 'cold', 'complaint', 'concerned', 'contact', 'cool', 'could', 'covid', 'create', 'customer', 'cya', 'damn', 'day', 'dear', 'delicious', 'desk', 'detail', 'devoured', 'did', 'disappointed', 'discount', 'discovered', 'disgusting', 'dish', 'dissatisfied', 'do', 'dont', 'eat', 'ew', 'eww', 'executive', 'extremely', 'family', 'feedback', 'fes

<ipython-input-9-92d58d4645aa>:80: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)
C:\Users\Mayekar\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
41/41 [==============================] - 5s 6ms/step - loss: 3.1200 - accuracy: 0.0634
Epoch 2/200
41/41 [==============================] - 0s 3ms/step - loss: 2.9710 - accuracy: 0.1463
Epoch 3/200
41/41 [==============================] - 0s 3ms/step - loss: 2.8408 - accuracy: 0.1756
Epoch 4/200
41/41 [==============================] - 0s 3ms/step - loss: 2.7151 - accuracy: 0.2390
Epoch 5/200
41/41 [==============================] - 0s 3ms/step - loss: 2.5089 - accuracy: 0.2732
Epoch 6/200
41/41 [==============================] - 0s 3ms/step - loss: 2.2638 - accuracy: 0.3512
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 2.0259 - accuracy: 0.4098
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 1.8830 - accuracy: 0.4390
Epoch 9/200
41/41 [==============================] - 0s 1ms/step - loss: 1.8187 - accuracy: 0.4488
Epoch 10/200
41/41 [==============================] - 0s 2ms/step - loss: 1.5719 - accuracy: 0.5659
Epoch 11/

41/41 [==============================] - 0s 2ms/step - loss: 0.0789 - accuracy: 0.9756
Epoch 83/200
41/41 [==============================] - 0s 2ms/step - loss: 0.0996 - accuracy: 0.9756
Epoch 84/200
41/41 [==============================] - 0s 2ms/step - loss: 0.0985 - accuracy: 0.9610
Epoch 85/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0689 - accuracy: 0.9756
Epoch 86/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1167 - accuracy: 0.9610
Epoch 87/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0747 - accuracy: 0.9805
Epoch 88/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0663 - accuracy: 0.9854
Epoch 89/200
41/41 [==============================] - 0s 3ms/step - loss: 0.1385 - accuracy: 0.9610
Epoch 90/200
41/41 [==============================] - 0s 4ms/step - loss: 0.0892 - accuracy: 0.9659
Epoch 91/200
41/41 [==============================] - 0s 2ms/step - loss: 0.1013 - accuracy: 0.9707
Epoch 92/200


41/41 [==============================] - 0s 4ms/step - loss: 0.0303 - accuracy: 0.9902
Epoch 163/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0529 - accuracy: 0.9805
Epoch 164/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0719 - accuracy: 0.9756
Epoch 165/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0380 - accuracy: 0.9854
Epoch 166/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0817 - accuracy: 0.9756
Epoch 167/200
41/41 [==============================] - 0s 4ms/step - loss: 0.0588 - accuracy: 0.9805
Epoch 168/200
41/41 [==============================] - 0s 5ms/step - loss: 0.0270 - accuracy: 0.9951
Epoch 169/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0383 - accuracy: 0.9902
Epoch 170/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0494 - accuracy: 0.9854
Epoch 171/200
41/41 [==============================] - 0s 3ms/step - loss: 0.0588 - accuracy: 0.9756
Epoc